# Attention is All You Need (NIPS 2017) 실습

트랜스포머 정리 노트: https://github.com/gjlee0802/natural-language-processing/blob/main/NLP/transformer/attention_is_all_you_need_summary.md


독일어를 영어로 번역하는 Machine Translation 구현, 데이터셋은 Multi30k 이용.

# 데이터 전처리(Preprocessing)

spacy 라이브러리: 문장의 토큰화, 태깅 등의 전처리 기능을 위한 라이브러리

영어와 독일어 전처리 모듈 설치

In [1]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

In [3]:
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

In [4]:
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i} : {token.text}")

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .


## 토큰화 함수 정의 (spacy의 토크나이저 이용)

In [5]:
# 독일어 문장을 토큰화 하는 함수
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]


In [6]:
import torch
print(torch.__version__)

2.1.0+cu121


In [7]:
!pip install torchtext==0.16.0
!pip install portalocker>=2.0.0

## 어휘집 Vocab 만들기

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

In [9]:
SRC_LANG = 'de'
TGT_LANG = 'en'

In [10]:
tokenizer = {}
tokenizer['en'] = tokenize_en
tokenizer['de'] = tokenize_de

In [11]:
# 토큰 목록을 생성하기 위한 헬퍼(helper) 함수
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANG: 0, TGT_LANG: 1}

    for i, datasample_tuple in enumerate(train_iter):
      yield tokenizer[language](datasample_tuple[language_index[language]])
    '''
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])
    '''

In [12]:
# 특수 기호(symbol)와 인덱스를 정의합니다
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# 토큰들이 어휘집(vocab)에 인덱스 순서대로 잘 삽입되어 있는지 확인합니다
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

vocab_transform = {} # 영어, 독일어에 대해서 torchtext의 Vocab 옵젝이 저장됨.

for ln in [SRC_LANG, TGT_LANG]:
  train_iter = Multi30k(split='train', language_pair=(SRC_LANG, TGT_LANG))
  val_iter = Multi30k(split='valid', language_pair=(SRC_LANG, TGT_LANG))
  test_iter = Multi30k(split='test', language_pair=(SRC_LANG, TGT_LANG))

  vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                  min_freq=2, # 최소 2번 이상 등장한 단어만을 선택
                                                  specials=special_symbols,
                                                  special_first=True)

In [13]:
for idx, (x, y) in enumerate(train_iter):
  if idx == 5:
    break

  print(f'[{idx}] \nx:{x} \ny:{y}')

[0] 
x:Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche. 
y:Two young, White males are outside near many bushes.
[1] 
x:Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem. 
y:Several men in hard hats are operating a giant pulley system.
[2] 
x:Ein kleines Mädchen klettert in ein Spielhaus aus Holz. 
y:A little girl climbing into a wooden playhouse.
[3] 
x:Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster. 
y:A man in a blue shirt is standing on a ladder cleaning a window.
[4] 
x:Zwei Männer stehen am Herd und bereiten Essen zu. 
y:Two men are at the stove preparing food.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [14]:
# ``UNK_IDX`` 를 기본 인덱스로 설정합니다. 이 인덱스는 토큰을 찾지 못하는 경우에 반환됩니다.
# 만약 기본 인덱스를 설정하지 않으면 어휘집(Vocabulary)에서 토큰을 찾지 못하는 경우
# ``RuntimeError`` 가 발생합니다.
for ln in [SRC_LANG, TGT_LANG]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [15]:
print('소스 언어의 Vocab(어휘집)')
for idx in range(20):
  print(f'[Vocab] index: {idx} | token: {vocab_transform[SRC_LANG].lookup_token(idx)}')

소스 언어의 Vocab(어휘집)
[Vocab] index: 0 | token: <unk>
[Vocab] index: 1 | token: <pad>
[Vocab] index: 2 | token: <bos>
[Vocab] index: 3 | token: <eos>
[Vocab] index: 4 | token: .
[Vocab] index: 5 | token: Ein
[Vocab] index: 6 | token: einem
[Vocab] index: 7 | token: in
[Vocab] index: 8 | token: ,
[Vocab] index: 9 | token: und
[Vocab] index: 10 | token: mit
[Vocab] index: 11 | token: auf
[Vocab] index: 12 | token: Mann
[Vocab] index: 13 | token: einer
[Vocab] index: 14 | token: Eine
[Vocab] index: 15 | token: ein
[Vocab] index: 16 | token: der
[Vocab] index: 17 | token: Frau
[Vocab] index: 18 | token: eine
[Vocab] index: 19 | token: die


In [16]:
print('타겟 언어의 Vocab(어휘집)')
for idx in range(20):
  print(f'[Vocab] index: {idx} | token: {vocab_transform[TGT_LANG].lookup_token(idx)}')

타겟 언어의 Vocab(어휘집)
[Vocab] index: 0 | token: <unk>
[Vocab] index: 1 | token: <pad>
[Vocab] index: 2 | token: <bos>
[Vocab] index: 3 | token: <eos>
[Vocab] index: 4 | token: a
[Vocab] index: 5 | token: .
[Vocab] index: 6 | token: A
[Vocab] index: 7 | token: in
[Vocab] index: 8 | token: the
[Vocab] index: 9 | token: on
[Vocab] index: 10 | token: is
[Vocab] index: 11 | token: and
[Vocab] index: 12 | token: man
[Vocab] index: 13 | token: of
[Vocab] index: 14 | token: with
[Vocab] index: 15 | token: ,
[Vocab] index: 16 | token: woman
[Vocab] index: 17 | token: are
[Vocab] index: 18 | token: to
[Vocab] index: 19 | token: Two


In [17]:
print(vocab_transform[TGT_LANG].lookup_indices(['A']))

[6]


In [18]:
print(len(vocab_transform[SRC_LANG]))
print(len(vocab_transform[TGT_LANG]))

8014
6191


In [19]:
BATCH_SIZE = 128

배치 생성 전처리 코드 추가

In [20]:
from torch.nn.functional import pad

src_pipeline = lambda x: vocab_transform[SRC_LANG].lookup_indices(tokenizer[SRC_LANG](x))
tgt_pipeline = lambda x: vocab_transform[TGT_LANG].lookup_indices(tokenizer[TGT_LANG](x))

def collate_batch(batch):

  bs_id = torch.tensor([BOS_IDX])
  eos_id = torch.tensor([EOS_IDX])

  src_list, tgt_list = [], []
  for (_srctext, _tgttext) in batch:
    processed_src = torch.cat(
        [
            bs_id,
            torch.tensor(
                src_pipeline(_srctext),
                dtype=torch.int64
            ),
            eos_id,
        ],
        0,
    )
    processed_tgt = torch.cat(
        [
            bs_id,
            torch.tensor(
                tgt_pipeline(_tgttext),
                dtype=torch.int64
            ),
            eos_id
        ],
        0,
    )
    src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    BATCH_SIZE - len(processed_src),
                ),
                value=PAD_IDX,
            )
    )
    tgt_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    BATCH_SIZE - len(processed_src),
                ),
                value=PAD_IDX,
            )
    )

  src = torch.stack(src_list)
  tgt = torch.stack(tgt_list)
  return src, tgt

In [21]:
from torch.utils.data import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import Dataset, DataLoader

def train_valid_split(train_iterator, split_ratio=0.8, seed=42):
    train_count = int(split_ratio * len(train_iterator))
    valid_count = len(train_iterator) - train_count
    generator = torch.Generator().manual_seed(seed)
    train_set, valid_set = random_split(
        train_iterator, lengths=[train_count, valid_count], generator=generator)
    return train_set, valid_set

# iterable type에서 map style로 변환해야 length check 가능
train_iter = to_map_style_dataset(train_iter)
valid_iter = to_map_style_dataset(val_iter)
#train_set, valid_set = train_valid_split(train_iter)

train_dataloader = DataLoader(
    train_iter, batch_size=BATCH_SIZE, shuffle=True, collate_fn = collate_batch)
valid_dataloader = DataLoader(
    valid_iter, batch_size=BATCH_SIZE, shuffle=True, collate_fn = collate_batch)

In [22]:
print(len(train_iter))

29001


In [23]:
print(len(valid_iter))

1015


train_dataloader를 돌며 Source에 대한 배치 데이터 출력해보기

# Transformer 활용 Seq2Seq 모델

torch에서 제공하는 Transformer을 사용하지 않고, transformer을 구현하여 활용해보자.

In [24]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Multi Head Attention

어텐션은 세가지 요소를 입력으로 받는다.
- 쿼리(queries)
- 키(keys)
- 값(values)

하이퍼 파라미터
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_heads: 헤드의 개수(scaled dot-product attention 개수)
- dropout_ratio: 드롭아웃 비율

In [25]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    # assert는 뒤의 조건이 True가 아니면 AssertError를 발생한다.
    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim # 임베딩 차원
    self.n_heads = n_heads # 헤드의 개수(서로 다른 어텐션 컨셉의 수)
    self.head_dim = hidden_dim // n_heads # 각 헤드에서의 임베딩 차원 = 전체 임베딩 차원을 헤드의 수로 나눈 값

    self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
    self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
    self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):
    batch_size = query.shape[0]

    # query: [batch_size, query_len, hidden_dim]
    # key: [batch_size, key_len, hidden_dim]
    # value: [batch_size, value_len, hidden_dim]

    # 각각 FC 레이어에 입력
    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    # Q: [batch_size, query_len, hidden_dim]
    # K: [batch_size, key_len, hidden_dim]
    # V: [batch_size, value_len, hidden_dim]

    # hidden_dim -> n_heads X head_dim 형태로 변형
    # after permute
    # Q: [batch_size, query_len, n_heads, head_dim] -> [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, key_len, n_heads, head_dim] -> [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, value_len, n_heads, head_dim] -> [batch_size, n_heads, value_len, head_dim]
    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

    # Q: [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]

    # Attention Energy 계산 (유사도 계산)
    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

    # 마스크를 사용할 경우
    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10) # 마스크 값이 0인 부분에 상당이 작은 값으로 채워준다.

    # 어텐션 스코어 계산: 각 단어에 대한 확률 값
    attention = torch.softmax(energy, dim=-1) # 소프트맥스로 정규화

    # attention: [batch_size, n_heads, query_len, key_len]

    # Scaled Dot-Product Attention을 계산
    x = torch.matmul(self.dropout(attention), V)

    # x: [batch_size, n_heads, query_len, head_dim]

    x = x.permute(0,2,1,3).contiguous()

    # x: [batch_size, query_len, n_heads, head_dim]

    # n_heads X head_dim -> hidden_dim 변형
    x = x.view(batch_size, -1, self.hidden_dim)

    # x: [batch_size, query_len, hidden_dim]

    x = self.fc_o(x)

    return x, attention


## Position-wise Feedforward
입력과 출력의 차원이 동일함.  
- encoder와 decoder의 각각의 layer는 fully connected feed-forward network를 포함하고 있음.  
- position 마다, 즉 개별 단어마다 적용되기 때문에 position-wise.  
- network는 두 번의 linear transformation과 activation function ReLU로 이루어져 있음(fc1 -> relu -> fc2).  

하이퍼 파라미터
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
- dropout_ratio: 드롭아웃 비율

In [26]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hidden_dim, pf_dim, dropout_ratio):
      super().__init__()

      self.fc_1 = nn.Linear(hidden_dim, pf_dim)
      self.fc_2 = nn.Linear(pf_dim, hidden_dim)

      self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, x):

    # x: [batch_size, seq_len, hidden_dim]

    x = self.dropout(torch.relu(self.fc_1(x)))

    # x: [batch_size, seq_len, pf_dim]

    x = self.fc_2(x)

    # x: [batch_size, seq_len, hidden_dim]

    return x

## Encoder 레이어
인코더는 아래의 인코더 레이어를 여러번 중첩하여 사용함.  
인코더 레이어의 입력과 출력의 차원이 같음.  


하이퍼 파라미터
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_heads: 헤드의 개수
- pf_dim: Feedforward 레이어(PositionwiseFeedforward)에서의 내부 임베딩 차원
- dropout_ratio: 드롭아웃 비율

In [27]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, src, src_mask):
    # src: [batch_size, src_len, hidden_dim]
    # src_mask: [batch_size, src_len]

    # Self Attention
    # 필요한 경우 마스크 행렬을 이용하여 어텐션할 단어 조절 가능
    _src, _ = self.self_attention.forward(src, src, src, src_mask) # params : query, key, value, mask

    # dropout, residual connection and layer norm
    # residual connection : feedforward를 거치기 전 입력 x를 feedforward를 거친 결과값에 더해주어 입력하는 것
    src = self.self_atten_layer_norm.forward(src + self.dropout(_src))

    # src: [batch_size, src_len, hidden_dim]

    # Position-wisd feedforward
    _src = self.posittionwise_feedforward.forward(src)

    # dropout, residual and layer norm
    src = self.ff_layer_norm(src + self.dropout(_src))

    # src: [batch_size, src_len, hidden_dim]

    return src

## Encoder

하이퍼 파라미터
- input_dim: 하나의 단어에 대한 원-핫 인코딩 차원
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_layers: 내부적으로 사용할 인코더 레이어의 개수
- n_heads: 헤드의 개수
- pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
- dropout_ratio: 드롭아웃 비율
- max_length: 문장 내 최대 단어 개수

In [28]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, src, src_mask):
    # src: [batch_size, src_len]
    # src_mask: [batch_size, src_len]

    batch_size = src.shape[0]
    src_len = src.shape[1]

    # unsqueeze는 특정 위치에 1인 차원을 추가함.
    # unsqueeze(0)는 첫번째 차원에 1인 차원을 추가함. [1 X src_len]
    # repeat 함수는 텐서를 반복 확장시켜줌.
    # repeat(batch_size, 1)은 [1 X src_len] 형태를 [batch_size, src_len] 차원으로 만들어줌.
    pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # pos: [batch_size, src_len]

    # 소스 문장의 임베딩과 위치 임베딩을 더함. (Positional Encoding)
    src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

    # src: [batch_size, src_len, hidden_dim]

    # 모든 인코더 레이어를 차례대로 거치며 순전파 수행
    for layer in self.encoder_layers:
      src = layer(src, src_mask)

    # src: [batch_size, src_len, hidden_dim]

    # 마지막 레이어의 출력 반환
    return src

## Decoder 레이어

하이퍼 파라미터
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_heads: 헤드의 개수
- pf_dim: Feedforward 레이어(PositionwiseFeedforward)에서의 내부 임베딩 차원
- dropout_ratio: 드롭아웃 비율

In [29]:
class DecoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # 인코더의 출력 값(enc_src)를 어텐션하는 구조
  def forward(self, trg, enc_src, trg_mask, src_mask):
    # trg: [batch_size, trg_len, hidden_dim]
    # enc_src: [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len, hidden_dim]
    # src_mask: [batch_size, src_len, hidden_dim]

    _trg, _ = self.self_attention.forward(trg, trg, trg, trg_mask) # params : query, key, value, mask

    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

    # trg: [batch_size, trg_len, hidden_dim]

    # positionwise feedforward
    _trg = self.positionwise_feedforward(trg)

    # dropout, residual and layer norm
    # residual connection : feedforward를 거치기 전 입력 x를 feedforward를 거친 결과값에 더해주어 입력하는 것
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    # trg: [batch_size, trg_len, hidden_dim]


    return trg, attention

## Decoder
원본 논문과 다르게 위치 임베딩(positional embedding)을 학습하는 형태(BERT와 같은 모던 트랜스포머 모델에서 사용되는 방식)로 구현함.  
  
소스 문장의 'pad' 토큰에 대해 마스크(MASK) 값을 0으로 설정함.  
  
Masked Decoder Self-Attention: 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크(MASK)를 사용함.
~~~
Masked Decoder Self-Attention : 디코더 파트에서 셀프 어텐션을 사용할 때는 각각의 출력 단어가 다른 모든 출력 단어를 참고하도록 하지는 않고, 앞쪽의 단어들만 참고하도록 함.
~~~

하이퍼 파라미터
- output_dim: 하나의 단어에 대한 원-핫 인코딩 차원
- hidden_dim: 하나의 단어에 대한 임베딩 차원
- n_layers: 내부적으로 사용할 인코더 레이어의 개수
- n_heads: 헤드의 개수
- pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
- dropout_ratio: 드롭아웃 비율
- max_length: 문장 내 최대 단어 개수

In [30]:
class Decoder(nn.Module):
  def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.decoder_layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.fc_out = nn.Linear(hidden_dim, output_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, trg, enc_src, trg_mask, src_mask):
    # trg: [batch_size, trg_len]
    # enc_src: [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    # unsqueeze는 특정 위치에 1인 차원을 추가함.
    # unsqueeze(0)는 첫번째 차원에 1인 차원을 추가함. [1 X trg_len]
    # repeat 함수는 텐서를 반복 확장시켜줌.
    # repeat(batch_size, 1)은 [1 X trg_len] 형태를 [batch_size, trg_len] 차원으로 만들어줌.
    pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # pos : [batch_size, trg_len]

    # 타겟 문장의 임베딩과 위치 임베딩을 더함. (Positional Encoding)
    trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

    # trg: [batch_size, trg_len, hidden_dim]

    # 모든 디코더 레이어를 거치며 순전파 수행
    for layer in self.decoder_layers:
      trg, attention = layer(trg, enc_src, trg_mask, src_mask) # 소스 마스크와 타겟 마스크 모두 사용

    # trg: [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    output = self.fc_out(trg)

    # output: [batch_size, trg_len, output_dim]

    return output, attention

## Transformer Model
입력이 들어왔을 때 앞서 정의한 Encoder와 Decoder을 거쳐 출력 문장을 생성함.

파라미터
- encoder: encoder 객체
- decoder: decoder 객체
- src_pad_idx: 소스 문장의 패딩 문자 인덱스
- trg_pad_idx: 타겟 문장의 패딩 문자 인덱스

In [31]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    #self.src_pad_idx = src_pad_idx
    #self.trg_pad_idx = trg_pad_idx
    self.padding_idx = src_pad_idx

    self.device = device
  '''
  def make_src_mask(self, src):

    # src: [batch_size, src_len]

    # unsqueeze를 사용하여 1번째 자리, 2번째 자리에 1인 차원을 추가한다.
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

    # src_mask: [batch_size, 1, 1, src_len]

    return src_mask

  def make_trg_mask(self, trg):

    # trg: [batch_size, trg_len]

    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

    # trg_pad_mask: [batch_size, 1, 1, trg_len]

    trg_len = trg.shape[1]

    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

    # trg_sub_mask: [trg_len, trg_len] # 정방 행렬 텐서

    trg_mask = trg_pad_mask & trg_sub_mask

    # trg_mask: [batch_size, 1, trg_len, trg_len]

    return trg_mask
  '''
  def make_padding_mask(self, q, k):
    # q,k의 size = (batch_size, seq_len)
    _, q_seq_len = q.size()
    _, k_seq_len = k.size()

    q = q.ne(self.padding_idx)  # padding token을 0, 나머지를 1로 만들어줌
    q = q.unsqueeze(1).unsqueeze(3) # (batch_size, 1, q_seq_len, 1)
    q = q.repeat(1,1,1,k_seq_len)   # (batch_size, 1, q_seq_len, k_seq_len)

    k = k.ne(self.padding_idx)
    k = k.unsqueeze(1).unsqueeze(2) # (batch_size, 1, 1, k_seq_len)
    k = k.repeat(1,1,q_seq_len,1)   # (batch_size, 1, q_seq_len, k_seq_len)

    # and 연산
    # (batch_size, 1, q_seq_len, k_seq_len)
    mask = q & k

    return mask
  def make_look_ahead_mask(self, tgt):
    _, seq_len = tgt.size()

    # torch.tril 함수를 사용하여 한칸씩 밀려나며 마스킹을 해줌
    # (seq_len, seq_len)
    mask = torch.tril(torch.ones(seq_len,seq_len)).type(torch.BoolTensor).to(self.device)

    return mask

  def forward(self, src, trg):
    # src: [batch_size, src_len]
    # trg: [batch_size, trg_len]

    '''
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    # src_mask: [batch_size, 1, 1, src_len]
    # trg_mask: [batch_size, 1, trg_len, trg_len]
    '''

    src_mask = self.make_padding_mask(src, src)                                 # padding_mask
    enc_dec_padding_mask = self.make_padding_mask(trg, src)                     # enc_dec_padding_mask
    trg_mask = self.make_padding_mask(trg, trg) * self.make_look_ahead_mask(trg)# look_ahead_mask

    enc_src = self.encoder(src, src_mask)

    # enc_src: [batch_size, src_len, hidden_dim]

    output, attention = self.decoder(trg, enc_src, trg_mask, enc_dec_padding_mask)

    # output: [batch_size, trg_len, otuput_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    return output, attention


## Training (학습)

### 하이퍼 파라미터 설정

In [32]:
INPUT_DIM = len(vocab_transform[SRC_LANG])
OUTPUT_DIM = len(vocab_transform[TGT_LANG])
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 모델 초기화

In [33]:
# 패딩 인덱스를 확인하기 위해 출력해본다.
print(vocab_transform[SRC_LANG].lookup_indices(['<pad>']))

[1]


In [34]:
SRC_PAD_IDX = vocab_transform[SRC_LANG].lookup_indices(['<pad>'])
TGT_PAD_IDX = vocab_transform[TGT_LANG].lookup_indices(['<pad>'])

# 인코더와 디코더 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX[0], TGT_PAD_IDX[0], device).to(device)

### 모델 가중치 파라미터 초기화

In [35]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,232,431 trainable parameters


In [36]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data) # 가중치를 Xavier 값으로 초기화

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(8014, 256)
    (pos_embedding): Embedding(100, 256)
    (encoder_layers): ModuleList(
      (0-2): 3 x EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=Fals

### 학습 및 평가 함수 정의

optimizer는 Adam optimizer 사용

In [37]:
import torch.optim as optim

lr = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 뒷 부분의 패딩에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

모델 학습 함수

In [38]:
print(len(train_dataloader))

227


In [39]:
def train(model, dataloader, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0

  for index, batch in enumerate(dataloader):
    src = batch[0]
    tgt = batch[1]

    optimizer.zero_grad()

    # 출력 단어의 마지막 인덱스()는 제외
    # 입력을 할 때는 부터 시작하도록 처리
    output, _ = model(src, tgt[:,:-1])

    # output: [배치 크기, tgt_len - 1, output_dim]
    # trg: [배치 크기, tgt_len]

    output_dim = output.shape[-1]

    output = output.contiguous().view(-1, output_dim)
    # 출력 단어의 인덱스 0()은 제외
    tgt = tgt[:,1:].contiguous().view(-1)

    # output: [배치 크기 * tgt_len - 1, output_dim]
    # tgt: [배치 크기 * tgt_len - 1]

    # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
    loss = criterion(output, tgt)
    loss.backward() # 기울기(gradient) 계산

    # 기울기(gradient) clipping 진행
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    # 파라미터 업데이트
    optimizer.step()

    # 전체 손실 값 계산
    epoch_loss += loss.item()

  return epoch_loss / len(dataloader)

In [41]:
# 모델 평가(evaluate) 함수
def evaluate(model, dataloader, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(dataloader):
            src = batch[0]
            tgt = batch[1]

            # 출력 단어의 마지막 인덱스()는 제외
            # 입력을 할 때는 부터 시작하도록 처리
            output, _ = model(src, tgt[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # tgt: [배치 크기, tgt_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0()은 제외
            tgt = tgt[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # tgt: [배치 크기 * tgt_len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, tgt)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

In [42]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [43]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

128
128
128


IndexError: index out of range in self